<a href="https://colab.research.google.com/github/NURKISH-BANU/movie-review-sentiment-analysis/blob/main/movie_review_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 118.9 MB/s eta 0:00:00


In [ ]:
# ==========================
# Step 0: Install missing libraries
# ==========================
!pip install shap
!pip install beautifulsoup4

# ==========================
# Step 1: Import Libraries
# ==========================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from bs4 import BeautifulSoup

# Sklearn libraries for ML
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, RocCurveDisplay

# SHAP for explainability (optional bonus)
import shap

# Save/load models
import joblib

# For file upload in Colab
from google.colab import files

import warnings
warnings.filterwarnings("ignore")
print("Libraries loaded successfully!")

# ==========================
# Step 2: Upload Dataset
# ==========================
uploaded = files.upload()  # Upload IMDB Dataset.csv

# Load dataset
df = pd.read_csv("IMDB Dataset.csv")
print("Shape of dataset:", df.shape)
print("\nMissing values:\n", df.isnull().sum())
df.head()

# ==========================
# Step 3: Check Sentiment Balance
# ==========================
sns.countplot(x='sentiment', data=df, palette='coolwarm')
plt.title("Sentiment Distribution")
plt.show()

print(df['sentiment'].value_counts())

# ==========================
# Step 4: Data Cleaning & Preprocessing
# ==========================
# Custom stopwords
custom_stopwords = set([
    'a','about','above','after','again','against','all','am','an','and','any','are',
    'as','at','be','because','been','before','being','below','between','both','but','by',
    'could','did','do','does','doing','down','during','each','few','for','from','further',
    'had','has','have','having','he','her','here','hers','herself','him','himself','his',
    'how','i','if','in','into','is','it','its','itself','just','me','more','most','my',
    'myself','no','nor','not','now','of','off','on','once','only','or','other','our',
    'ours','ourselves','out','over','own','same','she','should','so','some','such','than',
    'that','the','their','theirs','them','themselves','then','there','these','they','this',
    'those','through','to','too','under','until','up','very','was','we','were','what',
    'when','where','which','while','who','whom','why','will','with','you','your','yours',
    'yourself','yourselves'
])

# Cleaning function
def clean_review(review):
    review = BeautifulSoup(review, "html.parser").get_text()  # Remove HTML
    review = re.sub("[^a-zA-Z]", " ", review)               # Remove non-alphabetic
    review = review.lower()                                 # Lowercase
    words = [word for word in review.split() if word not in custom_stopwords]  # Remove stopwords
    return " ".join(words)

df['cleaned_review'] = df['review'].apply(clean_review)
df[['review','cleaned_review','sentiment']].head()

# ==========================
# Step 5: Convert Sentiment to Numeric
# ==========================
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

# ==========================
# Step 6: TF-IDF Vectorization
# ==========================
X = df['cleaned_review']
y = df['sentiment']

tfidf = TfidfVectorizer(max_features=5000)
X_tfidf = tfidf.fit_transform(X)

print("TF-IDF shape:", X_tfidf.shape)

# ==========================
# Step 7: Train/Test Split
# ==========================
X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf, y, test_size=0.2, random_state=42
)
print("Training data shape:", X_train.shape)
print("Testing data shape:", X_test.shape)

# ==========================
# Step 8: Function to Evaluate Models
# ==========================
def evaluate_model(model, model_name):
    # Train the model
    model.fit(X_train, y_train)

    # Predict on test data
    y_pred = model.predict(X_test)

    # Accuracy
    acc = accuracy_score(y_test, y_pred)
    print(f"{model_name} Accuracy: {acc:.4f}")

    # Detailed metrics
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, target_names=['Negative','Positive']))

    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Negative','Positive'], yticklabels=['Negative','Positive'])
    plt.title(f"{model_name} - Confusion Matrix")
    plt.show()

    return acc

# ==========================
# Step 8a: Logistic Regression
# ==========================
lr_model = LogisticRegression(max_iter=1000)
lr_acc = evaluate_model(lr_model, "Logistic Regression")

# ==========================
# Step 8b: Naive Bayes
# ==========================
nb_model = MultinomialNB()
nb_acc = evaluate_model(nb_model, "Naive Bayes")

# ==========================
# Step 8c: SVM
# ==========================
svm_model = LinearSVC()
svm_acc = evaluate_model(svm_model, "Support Vector Machine")

# ==========================
# Step 8d: Random Forest
# ==========================
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_acc = evaluate_model(rf_model, "Random Forest")

# ==========================
# Step 9: Compare Model Accuracies
# ==========================
results = pd.DataFrame({
    'Model': ['Logistic Regression', 'Naive Bayes', 'SVM', 'Random Forest'],
    'Accuracy': [lr_acc, nb_acc, svm_acc, rf_acc]
})
results = results.sort_values(by='Accuracy', ascending=False)
sns.barplot(x='Accuracy', y='Model', data=results, palette='viridis')
plt.title("Model Performance Comparison")
plt.show()
results

# ==========================
# Step 10: Hyperparameter Tuning (Logistic Regression)
# ==========================
param_grid = {
    'C': [0.1, 1, 10],
    'penalty': ['l2'],
    'solver': ['liblinear']
}
grid = GridSearchCV(LogisticRegression(), param_grid, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)
grid.fit(X_train, y_train)

print("Best Parameters:", grid.best_params_)
print("Best CV Accuracy:", grid.best_score_)

best_lr = grid.best_estimator_
final_acc = evaluate_model(best_lr, "Tuned Logistic Regression")

# ==========================
# Step 11: Save Model & TF-IDF Vectorizer
# ==========================
joblib.dump(best_lr, "sentiment_model.pkl")
joblib.dump(tfidf, "tfidf_vectorizer.pkl")
print("Model and vectorizer saved successfully!")

# ==========================
# Step 12: Predict New Review Example
# ==========================
def predict_sentiment(review_text):
    cleaned = clean_review(review_text)
    vectorized = tfidf.transform([cleaned])
    prediction = best_lr.predict(vectorized)[0]
    return "Positive 😊" if prediction == 1 else "Negative 😞"

sample_review = "The movie was absolutely fantastic and thrilling!"
print("Review:", sample_review)
print("Predicted Sentiment:", predict_sentiment(sample_review))


Libraries loaded successfully!


In [ ]:
import joblib
joblib.dump(best_lr, "sentiment_model.pkl")
joblib.dump(tfidf, "tfidf_vectorizer.pkl")

['tfidf_vectorizer.pkl']

In [ ]:
from google.colab import files
files.upload()


Saving download.png to download.png


{'download.png': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x04\x00\x00\x00\x04\x00\x08\x02\x00\x00\x00\xf0\x7f\xbc\xd4\x00\x00\x00\x89zTXtRaw profile type iptc\x00\x00\x08\x99M\x8c1\x0e\x021\x0c\x04\xfb\xbc\xe2\x9e\x908\xeb\xb5]S\xd1Q\xf0\x81\xbb\\"!!\x81\xf8\x7fA\xa0\xe0\x98iV[L:_\xae\xa7\xe5\xf9z\x8c\xdb\xbd\xa7\xe5\x0b\x91\xaaC\x10\xd83\xa6?\x8a\x97\x96\x85}\xae\x8a\x9d\x85\xa0J6c\x18\xe8\x92a\x1cSc\xfb\xfc(\x88\xa3#\xfa\xd7\xc9\xcd\xb7\xeaA\xdd\x00\xad\xb6F\xf1\xd5\xbai\x847\xefu0\xbd\x01C\xeb"\xa2\xee\xfdZc\x00\x00\x02\x87iTXtXML:com.adobe.xmp\x00\x00\x00\x00\x00<?xpacket begin="\xef\xbb\xbf" id="W5M0MpCehiHzreSzNTczkc9d"?> <x:xmpmeta xmlns:x="adobe:ns:meta/" x:xmptk="XMP Core 5.5.0"> <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"> <rdf:Description rdf:about="" xmlns:Iptc4xmpExt="http://iptc.org/std/Iptc4xmpExt/2008-02-29/" xmlns:photoshop="http://ns.adobe.com/photoshop/1.0/" Iptc4xmpExt:DigitalSourceFileType="http://cv.iptc.org/newscodes/digitalsourcetype/tr

In [ ]:
import os
# Check if files exist
print("Model exists:", os.path.exists("sentiment_model.pkl"))
print("Vectorizer exists:", os.path.exists("tfidf_vectorizer.pkl"))

Model exists: True
Vectorizer exists: True


In [ ]:
!pip install streamlit
!pip install pyngrok  # for exposing the app online

In [ ]:
!npm install -g localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴
added 22 packages in 3s
⠦
⠦3 packages are looking for funding
⠦  run `npm fund` for details
⠦npm notice
npm notice New major version of npm available! 10.8.2 -> 11.7.0
npm notice Changelog: https://github.com/npm/cli/releases/tag/v11.7.0
npm notice To update run: npm install -g npm@11.7.0
npm notice
⠦

In [ ]:
%%writefile app.py
import base64
import streamlit as st
import joblib
import re
from bs4 import BeautifulSoup
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import pandas as pd

# =========================
# Function: Background Image
# =========================
def add_bg_from_local(image_file):
    with open(image_file, "rb") as file:
        encoded_string = base64.b64encode(file.read()).decode()
    st.markdown(
        f"""
        <style>
        /* App Background */
        .stApp {{
            background-image: url("data:image/png;base64,{encoded_string}");
            background-size: cover;
            background-position: center;
            background-attachment: fixed;
            color: #FFFFFF;
        }}

        /* Title */
        h1 {{
            color: #FFD700 !important;
            text-shadow: 2px 2px 5px #000000;
        }}

        /* Subheaders */
        h2, h3, h4, h5 {{
            color: #FFFAFA !important;
            text-shadow: 1px 1px 4px #000000;
        }}

        /* Sidebar */
        .css-1d391kg {{
            color: #FFFFFF !important;
        }}

        /* Text Area */
        textarea {{
            background-color: rgba(255, 255, 255, 0.85) !important;
            color: #000000 !important;
            font-size: 16px !important;
            border-radius: 8px;
        }}

        /* Predict Button */
        .stButton button {{
            background-color: #ff4b4b;
            color: white;
            font-size: 18px;
            border-radius: 8px;
            padding: 10px 20px;
            border: none;
            box-shadow: 2px 2px 5px #000000;
        }}
        .stButton button:hover {{
            background-color: #e60000;
            color: white;
        }}

        /* Custom White Output Box */
        .custom-output {{
            color: #FFFFFF !important;
            font-size: 20px;
            font-weight: bold;
            background: rgba(0, 0, 0, 0.6);
            padding: 15px;
            border-radius: 8px;
            text-align: center;
            box-shadow: 2px 2px 6px #000;
        }}

        /* Warning / Success Boxes */
        .stSuccess {{
            background-color: rgba(0, 128, 0, 0.8) !important;
            color: white !important;
        }}
        .stError {{
            background-color: rgba(255, 0, 0, 0.8) !important;
            color: white !important;
        }}

        /* Footer */
        footer {{
            color: #FFFFFF;
            text-align: center;
            font-size: 14px;
        }}
        </style>
        """,
        unsafe_allow_html=True
    )

# Call the background function
add_bg_from_local("download.png")  # Ensure image is in same folder

# =========================
# Load Model & Vectorizer
# =========================
best_lr = joblib.load("sentiment_model.pkl")
tfidf = joblib.load("tfidf_vectorizer.pkl")

# =========================
# Custom Stopwords
# =========================
custom_stopwords = set([
    'a','about','above','after','again','against','all','am','an','and','any','are',
    'as','at','be','because','been','before','being','below','between','both','but','by',
    'could','did','do','does','doing','down','during','each','few','for','from','further',
    'had','has','have','having','he','her','here','hers','herself','him','himself','his',
    'how','i','if','in','into','is','it','its','itself','just','me','more','most','my',
    'myself','no','nor','not','now','of','off','on','once','only','or','other','our',
    'ours','ourselves','out','over','own','same','she','should','so','some','such','than',
    'that','the','their','theirs','them','themselves','then','there','these','they','this',
    'those','through','to','too','under','until','up','very','was','we','were','what',
    'when','where','which','while','who','whom','why','will','with','you','your','yours',
    'yourself','yourselves'
])

# =========================
# Clean Review Function
# =========================
def clean_review(review):
    review = BeautifulSoup(review, "html.parser").get_text()
    review = re.sub("[^a-zA-Z]", " ", review)
    review = review.lower()
    words = [word for word in review.split() if word not in custom_stopwords]
    return " ".join(words)

# =========================
# Predict Sentiment Function
# =========================
def predict_sentiment(review_text):
    cleaned = clean_review(review_text)
    vectorized = tfidf.transform([cleaned])
    prob = best_lr.predict_proba(vectorized)[0]  # Probability scores
    prediction = best_lr.predict(vectorized)[0]
    confidence = round(max(prob) * 100, 2)
    sentiment = "Positive 😊" if prediction == 1 else "Negative 😞"
    return sentiment, confidence

# =========================
# Streamlit Layout
# =========================
st.markdown(
    """
    <h1 style='text-align: center;'>🎬 Movie Review Sentiment Analyzer 🎥</h1>
    <p style='text-align: center; font-size:18px;'>Predict if a movie review is Positive or Negative with confidence level</p>
    """,
    unsafe_allow_html=True
)

# Sidebar sample test reviews
st.sidebar.title("Quick Test Reviews")
sample_reviews = {
    "Loved It ❤️": "I absolutely loved this movie! The story was amazing and the acting was phenomenal.",
    "Hated It 😡": "I hated this movie. The plot made no sense and the acting was terrible."
}

selected_sample = st.sidebar.radio("Choose a sample review:", list(sample_reviews.keys()))
if selected_sample:
    st.session_state['sample'] = sample_reviews[selected_sample]

# Text area for user input
review_input = st.text_area("Enter your review here:", st.session_state.get('sample', ""))

# =========================
# Prediction Button
# =========================
if st.button("Predict Sentiment"):
    if review_input.strip() == "":
        st.warning("⚠️ Please enter a review!")
    else:
        result, confidence = predict_sentiment(review_input)
        st.markdown(
            f"""
            <div class='custom-output'>
                Predicted Sentiment: {result} <br>
                Confidence: {confidence}%
            </div>
            """,
            unsafe_allow_html=True
        )

# =========================
# Word Cloud Visualization
# =========================
st.markdown("---")
st.subheader("📊 Sentiment Word Cloud")

uploaded_file = st.file_uploader("Upload IMDB dataset (optional) to see WordCloud", type=["csv"])
if uploaded_file:
    df = pd.read_csv(uploaded_file)
    if 'sentiment' in df.columns and 'review' in df.columns:
        df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})
        df['cleaned_review'] = df['review'].apply(clean_review)

        # Positive Word Cloud
        positive_text = " ".join(df[df['sentiment'] == 1]['cleaned_review'])
        st.write("**Positive Reviews Word Cloud**")
        wordcloud_pos = WordCloud(width=800, height=400, background_color='white').generate(positive_text)
        plt.imshow(wordcloud_pos, interpolation='bilinear')
        plt.axis("off")
        st.pyplot(plt)

        # Negative Word Cloud
        negative_text = " ".join(df[df['sentiment'] == 0]['cleaned_review'])
        st.write("**Negative Reviews Word Cloud**")
        wordcloud_neg = WordCloud(width=800, height=400, background_color='black', colormap='Reds').generate(negative_text)
        plt.imshow(wordcloud_neg, interpolation='bilinear')
        plt.axis("off")
        st.pyplot(plt)
    else:
        st.error("Uploaded CSV must have 'review' and 'sentiment' columns.")

# Footer
st.markdown(
    """
    <hr>
    <footer>Built with ❤️ using Streamlit | Powered by Logistic Regression</footer>
    """,
    unsafe_allow_html=True
)


Writing app.py


In [ ]:
!pip install pyngrok


In [ ]:
from pyngrok import ngrok
!ngrok config add-authtoken 32gxiaEjDGaSqbLRpRP7CZj17F2_7iwD7DtCnQjzBnPnuAoWH


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!streamlit run app.py --server.port 8501 &>/dev/null&

In [ ]:
from pyngrok import ngrok

# Kill any existing tunnels (to avoid conflicts)
ngrok.kill()

# Connect to Streamlit port
public_url = ngrok.connect(8501)
print("Public URL:", public_url)


Public URL: NgrokTunnel: "https://857d37d729e0.ngrok-free.app" -> "http://localhost:8501"
